# App Review Analysis: Evaluating Foundation Model Robustness

This notebook analyzes how different prompt formulations affect foundation models' interpretation of app store reviews. We'll evaluate model robustness across various tasks including sentiment analysis, feature request identification, and bug report detection.

## Project Structure

The project consists of the following components:

1. `data_loader.py`: Handles loading and preprocessing of app store review datasets
2. `prompt_generator.py`: Creates base prompts and variants for review analysis
3. `model_runner.py`: Executes foundation models with different prompts
4. `evaluator.py`: Measures consistency and accuracy across prompt variants
5. `visualizer.py`: Generates visualizations of model robustness metrics

Each module is designed to be modular and reusable for different types of review analysis tasks.

In [ ]:
import sys
from pathlib import Path

# Add the src directory to Python path
src_path = Path.cwd().parent / 'src'
sys.path.append(str(src_path))

# Import project modules
from utils.data_loader import AppReviewDataset

# Other imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

## Data Loading and Preprocessing

In this section, we'll load a sample dataset of app store reviews using our `AppReviewDataset` class. The dataset will be preprocessed to ensure consistency and remove any invalid entries.

In [ ]:
# Initialize the dataset loader
data_dir = Path.cwd().parent / 'data'
dataset = AppReviewDataset(data_dir)

# Load sample dataset (50 reviews)
reviews_df = dataset.get_reviews(n_samples=50)

# Display basic statistics
stats = dataset.get_statistics()
print("Dataset Statistics:")
for key, value in stats.items():
    print(f"{key}: {value}")

# Display first few reviews
print("\nSample Reviews:")
display(reviews_df.head())

## Data Validation

Let's validate our dataset to ensure it meets our requirements:
1. Each review should have non-empty text
2. Ratings should be within valid range (1-5)
3. Dates should be properly formatted
4. No duplicate reviews

In [ ]:
def validate_dataset(df: pd.DataFrame) -> dict:
    """Validate the dataset and return validation results."""
    results = {
        'total_reviews': len(df),
        'empty_reviews': df['text'].isna().sum(),
        'invalid_ratings': df[~df['rating'].between(1, 5)].shape[0],
        'invalid_dates': df['date'].isna().sum(),
        'duplicate_reviews': df.duplicated(subset=['text']).sum()
    }
    return results

# Run validation
validation_results = validate_dataset(reviews_df)
print("Validation Results:")
for key, value in validation_results.items():
    print(f"{key}: {value}")

## Prompt Generation and Analysis

Now we'll explore different ways to phrase prompts for analyzing app store reviews. We'll create multiple variants for each task (sentiment analysis, feature request detection, and bug report identification) to test model robustness.

In [ ]:
from utils.prompt_generator import PromptGenerator

# Initialize the prompt generator
prompt_gen = PromptGenerator()

# Display available tasks and styles
print("Available tasks:", prompt_gen.get_task_list())
print("Available prompt styles:", prompt_gen.get_style_list())

In [ ]:
# Let's explore prompt variants for a sample review
sample_review = reviews_df.iloc[0]['text']

# Generate prompts for each task
for task in prompt_gen.get_task_list():
    print(f"\n=== {task.upper()} ANALYSIS PROMPTS ===")
    variants = prompt_gen.get_prompt_variants(task, sample_review)
    
    for variant in variants:
        print(f"\nStyle: {variant['style']}")
        print("Prompt:")
        print(variant['prompt'])

As we can see above, we have different prompt variants for each analysis task. The variants differ in:

1. Formality level (formal vs casual)
2. Structure (free-form vs structured)
3. Level of detail in instructions
4. Output format specification

These variations will help us evaluate how sensitive the models are to prompt phrasing and structure.

## Model Execution and Result Collection

In this section, we'll run our foundation models (OpenAI and Hugging Face) with different prompt variants. We'll collect and store the results for later analysis of model robustness.

In [ ]:
import asyncio
from utils.model_runner import ModelRunner

# Initialize the model runner
model_runner = ModelRunner()

# Display available models
print("Available Models:")
for provider, models in model_runner.get_available_models().items():
    print(f"\n{provider.upper()}:")
    for model in models:
        print(f"- {model}")

In [ ]:
async def analyze_reviews_with_variants(reviews, n_samples=5):
    """Analyze a sample of reviews with different prompt variants."""
    results = []
    sample_reviews = reviews.sample(n=n_samples)
    
    for _, review in sample_reviews.iterrows():
        review_results = {'review_id': review['review_id'], 'text': review['text'], 'tasks': {}}
        
        for task in prompt_gen.get_task_list():
            variants = prompt_gen.get_prompt_variants(task, review['text'])
            task_results = []
            
            for variant in variants:
                result = await model_runner.analyze_review(
                    review['text'],
                    variant['prompt'],
                    task
                )
                task_results.append({
                    'style': variant['style'],
                    'prompt': variant['prompt'],
                    'result': result
                })
                
            review_results['tasks'][task] = task_results
        results.append(review_results)
    
    return results

In [ ]:
# Run analysis on a sample of reviews
print("Analyzing reviews with different prompt variants...")
results = await analyze_reviews_with_variants(reviews_df, n_samples=5)

# Save results for later analysis
results_dir = Path.cwd().parent / 'data' / 'results'
results_dir.mkdir(exist_ok=True)

with open(results_dir / 'analysis_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print(f"\nAnalysis complete. Results saved to {results_dir / 'analysis_results.json'}")

In [ ]:
# Preview results for one review
print("\nSample Results Preview:")
sample_result = results[0]
print(f"Review: {sample_result['text']}")

for task, task_results in sample_result['tasks'].items():
    print(f"\n=== {task.upper()} ANALYSIS ===")
    for variant_result in task_results:
        print(f"\nStyle: {variant_result['style']}")
        if 'openai' in variant_result['result']:
            print("OpenAI:", variant_result['result']['openai']['response'])
        if 'huggingface' in variant_result['result']:
            print("HuggingFace:", variant_result['result']['huggingface']['predictions'])

## Model Robustness Evaluation

Now we'll evaluate the robustness of our models by analyzing:
1. Consistency across prompt variants
2. Agreement between different models
3. Response stability
4. Per-task and overall metrics

In [ ]:
from utils.evaluator import ModelEvaluator

# Initialize the evaluator
evaluator = ModelEvaluator()

# Load results from file
results_path = Path.cwd().parent / 'data' / 'results' / 'analysis_results.json'
with open(results_path) as f:
    analysis_results = json.load(f)

# Run evaluation
print("Evaluating model robustness...")
evaluation_results = evaluator.evaluate_results(analysis_results)

In [ ]:
# Display overall metrics
print("\nOverall Robustness Metrics:")
for metric, value in evaluation_results['overall_metrics'].items():
    print(f"{metric}: {value:.3f}")

# Create heatmap of task-specific metrics
metrics_data = []
for task, task_metrics in evaluation_results['per_task_metrics'].items():
    for metric, value in task_metrics.items():
        metrics_data.append({
            'Task': task,
            'Metric': metric,
            'Value': value
        })

metrics_df = pd.DataFrame(metrics_data)
metrics_pivot = metrics_df.pivot(index='Task', columns='Metric', values='Value')

plt.figure(figsize=(12, 6))
sns.heatmap(metrics_pivot, annot=True, cmap='YlGnBu', fmt='.3f')
plt.title('Model Robustness Metrics by Task')
plt.tight_layout()
plt.show()

In [ ]:
# Analyze per-review metrics
print("\nPer-Review Analysis:")
for review_id, review_metrics in evaluation_results['per_review_metrics'].items():
    review_text = next(
        r['text'] for r in analysis_results if str(r['review_id']) == review_id
    )
    print(f"\nReview {review_id}: {review_text[:100]}...")
    
    for task, metrics in review_metrics.items():
        print(f"  {task}: consistency = {metrics['consistency']:.3f}")

### Key Findings

1. **Prompt Variation Consistency**: Measures how consistent model responses are across different prompt phrasings
2. **Model Agreement**: Shows how well OpenAI and Hugging Face models agree on their predictions
3. **Response Stability**: Indicates how stable the responses are when prompt style changes

These metrics help us understand how robust the models are to changes in prompt formulation.

## Visualization of Results

Let's create various visualizations to better understand the model robustness analysis:

In [ ]:
from utils.visualizer import ModelVisualizer

# Initialize the visualizer
vis_dir = Path.cwd().parent / 'data' / 'visualizations'
visualizer = ModelVisualizer(vis_dir)

# Create visualizations
print("Generating visualizations...")

# 1. Prompt impact heatmaps for each task
for task in ['sentiment', 'feature_request', 'bug_report']:
    visualizer.create_prompt_impact_heatmap(analysis_results, task)
    
# 2. Model agreement plot
visualizer.plot_model_agreement(analysis_results)

# 3. Consistency matrix
visualizer.plot_consistency_matrix(evaluation_results)

# 4. Robustness radar plot
visualizer.plot_robustness_radar(evaluation_results)

print(f"\nVisualizations saved to {vis_dir}")

### Interpretation of Visualizations

1. **Prompt Impact Heatmaps**: Show how different prompt styles affect response characteristics for each task
2. **Model Agreement Plot**: Visualizes the level of agreement between OpenAI and Hugging Face models
3. **Consistency Matrix**: Displays the consistency of model behavior across different tasks and metrics
4. **Robustness Radar**: Provides a high-level view of model robustness across multiple dimensions

These visualizations help identify:
- Which prompt styles lead to more consistent results
- Where models tend to agree or disagree
- Tasks that are more sensitive to prompt variations
- Overall robustness patterns